In [ ]:
!pip install transformers

In [ ]:
"""
library:
transformers: consists of thousands of pretrained models to perform tasks on different modalities such as text, vision, and audio.
Here we are using the library for text summarization 


pipeline:
pipelines are objects that abstract most of the complex code from the library
There are two categories of pipeline abstractions to be aware about:
~pipeline() which is the most powerful object encapsulating all other pipelines.
~other task-specific pipelines: SummarizationPipeline, TextClassificationPipeline, etc.


BartConfig:
This is the configuration class to store the configuration of a BartModel. 
It is used to instantiate a BART model according to the specified arguments, defining the model architecture.

BartTokenizer
Constructs a BART tokenizer.
This tokenizer has been trained to treat spaces like parts of the tokens so a word will
be encoded differently whether it is at the beginning of the sentence (without space) or not.


BartForConditionalGeneration:
The BART Model with a language modeling head.
This model is a PyTorch torch.nn.Module sub-class.
"""

In [1]:
#ARTICLE1 contains 369 words
#summarization max words limit is set to 150 words


ARTICLE1 = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""


#ARTICLE2 contains 188 words
#summarization max words limit is set to 60 words


ARTICLE2="""Videos that say approved vaccines are dangerous and cause autism, cancer or infertility are among those 
that will be taken down, the company said.  The policy includes the termination of accounts of anti-vaccine influencers.  
Tech giants have been criticised for not doing more to counter false health information on their sites.  
In July, US President Joe Biden said social media platforms were largely responsible for people's scepticism in getting 
vaccinated by spreading misinformation, and appealed for them to address the issue.  YouTube, which is owned by Google, 
said 130,000 videos were removed from its platform since last year, when it implemented a ban on content spreading misinformation 
about Covid vaccines.  In a blog post, the company said it had seen false claims about Covid jabs "spill over into misinformation about vaccines 
in general". The new policy covers long-approved vaccines, such as those against measles or hepatitis B.  "We're expanding our medical 
misinformation policies on YouTube with new guidelines on currently administered vaccines that are approved and confirmed to be safe and
 effective by local health authorities and the WHO," the post said, referring to the World Health Organization."""

#1. Using pipeline

In [2]:
from transformers import pipeline

def text_summarization_pipeline(article,max_len):
  summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
  return summarizer(article, max_length=max_len, min_length=30, do_sample=False)[0]['summary_text']


In [3]:
text_summarization_pipeline(ARTICLE1,150)

'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'

In [4]:
text_summarization_pipeline(ARTICLE2,60)

'Videos that say approved vaccines are dangerous and cause autism, cancer or infertility are among those that will be taken down. YouTube, which is owned by Google, said 130,000 videos were removed from its platform since last year.'

#2.  Using BartTokenizer, BartForConditionalGeneration, BartConfig

In [7]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

def text_summarization_BART(article,max_len):
  
  inputs = tokenizer([article], return_tensors='pt')
  #return_tensors: if set, return tensors instead of list of python integers
  # pt: return PyTorch torch.Tensor objects.

  summary_ids = model.generate(inputs['input_ids'], max_length=max_len, early_stopping=False)

  return [tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids][0]

In [8]:
text_summarization_BART(ARTICLE1,150)

'Liana Barrientos, 39, is charged with two counts of offering a false instrument for filing in the first degree. In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men, and at one time, she was married to eight men at once. If convicted, she faces up to four years in prison.'

In [9]:
text_summarization_BART(ARTICLE2,60)

'Videos that say approved vaccines are dangerous and cause autism, cancer or infertility are among those that will be taken down. The policy includes the termination of accounts of anti-vaccine influencers. YouTube, which is owned by Google, said 130,000 videos were removed from its platform'